Using multiple layer LSTMs for classification exploration


In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, LSTM,Embedding
from keras.models import Sequential
from keras.initializers import Constant
import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 200
batch_size = 128

In [2]:
wars=pd.read_csv("/floyd/home/Data/Wars.csv", names=['Body'], header=0)
potter=pd.read_csv("/floyd/home/Data/Potter.csv", names=['Body'], header=0)

In [4]:
wars['label'] =0
potter['label'] =1

In [5]:
wars.head(3)

,Body,label
0,"<p>If the Jedi Order had arrested Palpatine, w...",0
1,"<p>Were there any active, widespread, common, ...",0
2,"<p>I visited a <a href=""https://thesffblog.com...",0


In [6]:
def clean_docs(doc1, doc2):
    doc1['Body'] = doc1['Body'].replace(r'\\n',' ', regex=True)
    doc2['Body'] = doc2['Body'].replace(r'\\n',' ', regex=True)
    doc1['Body'] = doc1['Body'].replace(r'\n',' ', regex=True)
    doc2['Body'] = doc2['Body'].replace(r'\n',' ', regex=True)
    doc1['Body'] = doc1['Body'].replace(r'\\r',' ', regex=True)
    doc2['Body'] = doc2['Body'].replace(r'\\r',' ', regex=True)
    doc1['Body'] = doc1['Body'].replace(r'\r',' ', regex=True)
    doc2['Body'] = doc2['Body'].replace(r'\r',' ', regex=True)
    doc1['Body'] = doc1['Body'].str.replace('<[^<]+?>', '')
    doc2['Body'] = doc2['Body'].str.replace('<[^<]+?>', '')
    
    return (doc1, doc2)

In [7]:
wars, potter = clean_docs(wars, potter)

In [8]:
frames = [wars, potter]
all_data = pd.concat(frames)

In [9]:
all_data.Body = all_data.Body.astype(str)

In [10]:
all_data = all_data.sample(frac=1)

In [11]:
all_data = all_data[all_data.Body != 'nan']

In [12]:
texts=[]
for item in all_data['Body']:
    texts.append(item)

In [13]:
labels=[]
for item in all_data['label']:
    labels.append(item)

In [14]:
texts[987]

'At the end of Return of the Jedi, Anakin killed the Emperor, bringing balance to the Force and completing the ancient Jedi prophecy. There was peace in the galaxy and Luke and Leia were reunited.  I watched the Star Wars in the mid-2000\'s (I was born in the late 90\'s) and before I saw George Lucas\' quote:     Star Wars is a saga of Good vs. Evil, divided into nine parts   I thought the saga ended with the Return of the Jedi.  Now after watching The Force Awakens, I wonder: will the Star Wars saga never end? Because no matter how "perfect" or satisfying the end is (in the latest trilogy), 30 or 50 or 100 years later the Dark Side will rise again, and the saga will never actually end? '

In [15]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences =pad_sequences(sequences, maxlen=maxlen)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(sequences,labels, test_size = 0.12, random_state = 36)

In [17]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128, dropout=.5, recurrent_dropout=.2, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
model.fit(sequences, labels,
          batch_size=batch_size,
          epochs=3,
          validation_split=.1)

Train on 10854 samples, validate on 1207 samples
Epoch 1/3
10854/10854 [==============================] - 137s 13ms/step - loss: 0.2777 - acc: 0.8512 - val_loss: 0.0259 - val_acc: 0.9917
Epoch 2/3
10854/10854 [==============================] - 83s 8ms/step - loss: 0.0449 - acc: 0.9866 - val_loss: 0.0216 - val_acc: 0.9942
Epoch 3/3
10854/10854 [==============================] - 84s 8ms/step - loss: 0.0462 - acc: 0.9902 - val_loss: 0.0429 - val_acc: 0.9925
